# Agentic Retrieval‑Augmented Generation with LangGraph (Local LLM)

This notebook demonstrates an **agentic RAG pipeline** using:
- **LangGraph** as the agent orchestration framework
- **FAISS** for semantic retrieval
- A **local causal LLM** (`gpt2‑medium`) via HuggingFace’s `AutoModelForCausalLM`

It includes **token budgeting**, **time budgeting**, and **loop avoidance** techniques.

In [1]:
# Install dependencies
!pip install langgraph langchain sentence-transformers faiss-cpu transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 16.9 MB/s eta 0:00:00


In [6]:
import time
import numpy as np
import faiss
from typing import TypedDict, List

from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Setup local LLM and embeddings (no API cost)
MODEL_NAME = 'gpt2-medium'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

embed_model = SentenceTransformer('all-MiniLM-L6-v2')

## Build Retrieval Index (FAISS)

In [7]:
# Doc corpus (example)
documents = [
    "RAG agents combine retrieval with generative LLMs.",
    "FAISS is a library for fast vector similarity search.",
    "Token budgeting keeps processing within limits.",
    "Time budgeting limits how long a step can run.",
    "LangGraph orchestrates agentic workflows."
]

# Create embeddings and FAISS index
doc_embeddings = embed_model.encode(documents)
dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(doc_embeddings))

In [8]:
def retrieve_docs(query: str, top_k: int = 3) -> List[str]:
    q_emb = embed_model.encode([query])
    D, I = index.search(np.array(q_emb), top_k)
    return [documents[i] for i in I[0]]

In [9]:
def generate_local(prompt: str, max_tokens: int = 100) -> str:
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        pad_token_id=tokenizer.eos_token_id  # prevent warnings
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

## Define LangGraph State

Our state will keep track of the question, retrieved context, answers, retry count, and history to avoid loops.

In [10]:
class AgentState(TypedDict):
    question: str
    context: List[str]
    answer: str
    retry_count: int
    history: List[str]

# Budgets
MAX_STEPS = 5
MAX_TOTAL_TOKENS = 400
MAX_STEP_TIME = 8  # seconds

total_tokens = 0

## Nodes (LangGraph)

1. **retrieve_node:** Retrieves documents based on the question.
2. **generate_node:** Generates an answer using local LLM.
3. **decide_next:** Determines whether to continue looping or stop.

In [16]:
def retrieve_node(state: AgentState) -> AgentState:
    start = time.time()
    docs = retrieve_docs(state['question'], top_k=3)
    elapsed = time.time() - start
    if elapsed > MAX_STEP_TIME:
        state['context'] = []
    else:
        state['context'] = docs
    return state  # always return state dict

def generate_node(state: AgentState) -> AgentState:
    global total_tokens
    context = '\n'.join(state['context'])
    prompt = f"Context: {context}\nQ: {state['question']}\nA:"
    start = time.time()
    answer = generate_local(prompt, max_tokens=100)
    elapsed = time.time() - start
    if elapsed > MAX_STEP_TIME:
        answer = "(generation timed out)"
    # Token budgeting
    toks = len(tokenizer.encode(answer))
    total_tokens += toks
    state['answer'] = answer
    return state

def decide_next_step(state: AgentState) -> AgentState:
    # Increment retry
    state['retry_count'] += 1

    # Infinite loop detection using cosine similarity
    ans_emb = embed_model.encode([state['answer']])
    repeated = False
    for prev in state['history']:
        prev_emb = embed_model.encode([prev])
        sim = np.dot(ans_emb, prev_emb.T) / (np.linalg.norm(ans_emb)*np.linalg.norm(prev_emb))
        if sim > 0.9:
            repeated = True
            break

    # Save to history
    state['history'].append(state['answer'])

    # Decide done flag
    state['done'] = False
    if repeated or state['retry_count'] >= MAX_STEPS or total_tokens >= MAX_TOTAL_TOKENS:
        state['done'] = True

    return state


## Build Graph


In [19]:
graph = StateGraph(AgentState)

# Register nodes
graph.add_node("retrieve_node", retrieve_node)
graph.add_node("generate_node", generate_node)
graph.add_node("decide_node", decide_next_step)

# Use string keys for start/end
graph.add_edge(START, "retrieve_node")
graph.add_edge("retrieve_node", "generate_node")
graph.add_edge("generate_node", "decide_node")
graph.add_edge("decide_node", END)

# Compile graph
compiled = graph.compile()

## Invoke the Agent


In [20]:
initial_state = {
    'question': "Explain RAG agents in simple terms.",
    'context': [],
    'answer': '',
    'retry_count': 0,
    'history': []
}

result = compiled.invoke(initial_state)

print("Final Answer:", result['answer'])
print("Context Retrieved:", result['context'])


Final Answer: (generation timed out)
Context Retrieved: ['RAG agents combine retrieval with generative LLMs.', 'LangGraph orchestrates agentic workflows.', 'FAISS is a library for fast vector similarity search.']


## 📌 What This Notebook Implements
- **Agentic workflow:** Your agent decides what to do next after retrieval and generation, without hard‑coding a single linear chain.:contentReference[oaicite:1]{index=1}
- **Token budgeting:** Track total tokens generated and stop when exceeded.
- **Time budgeting:** Bound how long each step can run.
- **Infinite loop prevention:** Use semantic similarity between answers to stop loops.
- **Fully local execution:** Runs entirely without external API calls or internet dependencies.